In [ ]:
from google.colab import drive
from google.colab import files
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/My Drive/Diploma/second_try/Code/other_language/german_rev.csv")
df['title'] = df['page_title']
df.head(2)


,rev_page,page_title,date,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs,day_after_revs,day_after_2_revs,title
0,13,Liste_von_Autoren/A,20220601,0.0,0.0,0.0,0.0,25.0,1.0,1.0,0.0,Liste_von_Autoren/A
1,37,Liste_von_Autoren/Z,20220601,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,Liste_von_Autoren/Z


# add views features

In [ ]:
!pip install pageviewapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.7 MB/s eta 0:00:00


In [1]:
from numpy import around
import pageviewapi
from datetime import datetime, timedelta
import swifter
import pandas as pd


# https://github.com/Comhttps://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles=2021%E2%80%932022_Boulder_County_fires&rvprop=ids&rvlimit=1&rvstart=2022-01-01T00%3A00%3A00.010Zmonists/pageview-api
def get_n_views(title, from_, to_):
  try:
    res = sum([i['views'] for i in pageviewapi.per_article('en.wikipedia', title, from_, to_,
                        access='all-access', agent='all-agents', granularity='daily')['items']])
  except:
    #print(title, from_, to_)
    res = -1
  return res
  
def get_views_features(title, day):
  before_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=2), '%Y%m%d')
  after_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=2), '%Y%m%d')
  before_6 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=6), '%Y%m%d')

  around = get_n_views(title, before_2, after_2)
  before = get_n_views(title, before_6, before_2)

  boost = around/before

  prev_day = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=1), '%Y%m%d')
  week_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=7), '%Y%m%d')
  month_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=30), '%Y%m%d')
  year_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=365), '%Y%m%d')

  day_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=1), '%Y%m%d')
  # week_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=7), '%Y%m%d')
  # month_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=30), '%Y%m%d')

  same_day_views = get_n_views(title, day, day)
  prev_day_views = get_n_views(title, prev_day, day)
  week_before_views = get_n_views(title, week_before, day)
  month_before_views = get_n_views(title, month_before, day)
  year_before_views = get_n_views(title, year_before, day)

  day_after_views = get_n_views(title, day, day_after)
  # week_after_views = get_n_views(title, day, week_after)
  # month_after_views = get_n_views(title, day, month_after)

  return [title, day,  around, before, boost, same_day_views, prev_day_views, week_before_views, month_before_views, 
          year_before_views, day_after_views#, week_after_views, month_after_views
          ]


In [ ]:
df['date'] = df['date'].astype(str)

In [ ]:
len(df)/2

In [ ]:
views_features = df[:8448].swifter.apply(lambda x: get_views_features(x['title'], x['date']), axis=1)

In [ ]:
views_features

In [ ]:
views_features_df = pd.DataFrame(views_features.tolist(), columns=["title", "day", "around", "before", "boost", "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "day_after_views"])

In [ ]:
views_features_df.to_csv('german_rev_views_2022_1.csv') 
files.download('german_rev_views_2022_1.csv')

In [ ]:
views_features = df[10000:].swifter.apply(lambda x: get_views_features(x['title'], x['date']), axis=1)

In [ ]:
views_features_df = pd.DataFrame(views_features.tolist(), columns=["title", "day", "around", "before", "boost", "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "day_after_views"])
views_features_df.to_csv('german_rev_views_2022_2.csv') 
files.download('german_rev_views_2022_2.csv')

# Add topic features

In [ ]:
import json
import swifter
import requests

def get_topic(title):
  try:
    url = f'https://wikipedia-topic.wmcloud.org/api/v1/topic?lang=en&title={title}'
    r = requests.get(url).text
    r = json.loads(r)['results'][0] # score, topic
    a, b = r['score'], r['topic']
  except:
    a, b = -1, -1
  return title, a, b

a = "Prime_Minister_of_Haiti"
get_topic(a)

('Prime_Minister_of_Haiti',
 0.8670457601547241,
 'Geography.Regions.Americas.Central_America')

In [ ]:
len(set(df['title']))

16567

In [ ]:
category = df[:8448].swifter.apply(lambda x: get_topic(x['title']), axis=1) 

Pandas Apply:   0%|          | 0/8448 [00:00<?, ?it/s]

In [ ]:

category.to_csv(f'german_category_1.csv')
files.download(f'german_category_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
category = df[8448:].swifter.apply(lambda x: get_topic(x['title']), axis=1) 

Pandas Apply:   0%|          | 0/8449 [00:00<?, ?it/s]

In [ ]:

category.to_csv(f'german_category_2.csv')
files.download(f'german_category_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]


# Revision features are loaded from Quarry (it's faster)